In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"stephenkiilu","key":"da68981283aa1159e5caf2a0d1646417"}'}

In [ ]:
# create a temp directory called '.kaggle'
!mkdir ~/.kaggle

# copy the newly-uploaded file to that directory
! cp kaggle.json ~/.kaggle/

# change permission of the file
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list

ref                                                            title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
datasets/muratkokludataset/date-fruit-datasets                 Date Fruit Datasets                              408KB  2022-04-03 09:25:39            799        201  0.9375           
datasets/piterfm/2022-ukraine-russian-war                      2022 Ukraine Russia War                            2KB  2022-04-16 14:04:28           8783        489  1.0              
datasets/muratkokludataset/acoustic-extinguisher-fire-dataset  Acoustic Extinguisher Fire Dataset               621KB  2022-04-02 22:59:36             56        181  0.9375           
datasets/kamilpytlak/personal-key-indicators-of-heart-disease  Personal Key Indi

In [ ]:
import os

In [ ]:
!mkdir Cassava_Disease_Classification
os.chdir('Cassava_Disease_Classification')

In [ ]:
!pip install --upgrade pip
!pip install kaggle==1.5.6

In [ ]:
!kaggle competitions download -c ammi-2021-convnets

In [ ]:
!unzip ammi-2021-convnets.zip

In [ ]:
!pip install ttach
!pip install git+https://github.com/gbaydin/hypergradient-descent.git
!pip install pretrainedmodels

  Cloning https://github.com/gbaydin/hypergradient-descent.git to /tmp/pip-req-build-vvzq9gza
  Running command git clone --filter=blob:none --quiet https://github.com/gbaydin/hypergradient-descent.git /tmp/pip-req-build-vvzq9gza
  Resolved https://github.com/gbaydin/hypergradient-descent.git to commit 020d6080c4cedfbc88d5cdb7a2a53f92b34c2b16
  Preparing metadata (setup.py) ... done
  Created wheel for hypergrad: filename=hypergrad-0.1-py3-none-any.whl size=8199 sha256=fa1df9a2f3d4f6bc30c9973e1b1dc78a316e1462ca1fb451f5fed3ccca791d1f
  Stored in directory: /tmp/pip-ephem-wheel-cache-ktbqji7_/wheels/fa/d5/8a/dc75fcd964cc54c2d95c3989c7b3382cd31c71bf1ec7d34750
Successfully built hypergrad
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=5ae733ad903c0da3e60de2aadac71adb65416be06c85feeaac6e44c6176e1483
  Stored in dir

In [ ]:
#Imports
import os
import sys
import glob
import torch
import torchvision

import numpy    as np
import datetime as dt
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot   as plt

from PIL               import Image
from torch.utils.data  import Dataset
from torch.autograd    import Variable
from torch.optim       import lr_scheduler

from torch.utils.data  import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision       import transforms, datasets, models
from os                import listdir, makedirs, getcwd, remove
from os.path           import isfile, join, abspath, exists, isdir, expanduser


import pandas as pd

from hypergrad import SGDHD, AdamHD

import pretrainedmodels


import ttach as tta


%matplotlib inline

In [ ]:
print(pretrainedmodels.model_names)

['fbresnet152', 'bninception', 'resnext101_32x4d', 'resnext101_64x4d', 'inceptionv4', 'inceptionresnetv2', 'alexnet', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'inceptionv3', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19_bn', 'vgg19', 'nasnetamobile', 'nasnetalarge', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn131', 'dpn107', 'xception', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'cafferesnet101', 'pnasnet5large', 'polynet']


In [ ]:
data_path = "../Cassava_Disease_Classification/"
train_path = join(data_path, "train/train")
test_path = join(data_path,"test/test")
extraimage_path = join(data_path, "extraimages/extraimages")

In [ ]:
NAME = "SUBMISSION"

MODEL_NAME1 = 'resnet152' # could be fbresnet152 or inceptionresnetv2

DIM_1 = 550
DIM_2 = 224
DIM_TEST_1 = 550
DIM_TEST_2 = 224

BATCH_SIZE = 8

NUM_EPOCHS1 = 5

random_seed = 42
shuffle_dataset = True
validation_split = .1

In [ ]:
# Transformations for both the training and testing data
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

# Do data transforms here, Try many others
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.Resize(DIM_1),
                                       transforms.RandomCrop(DIM_2),
                                       transforms.RandomHorizontalFlip(0.3),
                                       transforms.RandomVerticalFlip(0.3),
                                       transforms.ToTensor(),
                                       transforms.RandomErasing(0.1),
                                       transforms.Normalize(mean=mean, std=std)])

test_transforms = transforms.Compose([ transforms.Resize(DIM_TEST_1),
                                      transforms.CenterCrop(DIM_TEST_1),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=mean, std=std)])

In [ ]:


class CassavaDataset(Dataset):
    def __init__(self, path, dim, transform=None):
        self.classes = os.listdir(path)
        self.path = [f"{path}/{className}" for className in self.classes]
        self.file_list = [glob.glob(f"{x}/*") for x in self.path]
        self.transform = transform
        self.dim = dim

        self.targets = []
        

        files = []
        for i, className in enumerate(self.classes):
            for fileName in self.file_list[i]:
                files.append([i, className, fileName])
                self.targets.append(i)
                
        self.file_list = files
        files = None

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        fileName = self.file_list[idx][2]
        classCategory = self.file_list[idx][0]
        im = Image.open(fileName)
        if self.transform:
            im = self.transform(im)
            
        return im.view(3, self.dim, self.dim), classCategory

In [ ]:
class CassavaTestDataset(Dataset):
    def __init__(self, path, dim, transform=None):
        self.classes = os.listdir(path)
        self.path = [f"{path}/{className}" for className in self.classes]
        self.file_list = [glob.glob(f"{x}/*") for x in self.path]
        self.transform = transform
        self.indices = []
        self.dim=dim

        files = []
        for i, className in enumerate(self.classes):
            for fileName in self.file_list[i]:
                files.append([i, className, fileName])
                self.indices.append(fileName.split("/")[-1])
        self.file_list = files
        files = None

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        fileName = self.file_list[idx][2]
        index = self.file_list[idx][2]
        im = Image.open(fileName)
        if self.transform:
            im = self.transform(im)
            
        return im.view(3, self.dim, self.dim), index

In [ ]:
train_data = CassavaDataset(train_path, dim=DIM_2, transform=train_transforms)
test_data = CassavaTestDataset(test_path, dim=DIM_TEST_1, transform=test_transforms)

In [ ]:
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]

In [ ]:
# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)


train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE,
                                             sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE,
                                             sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)

In [ ]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
def test(model, data_loader):
    """Measures the accuracy of a model on a data set.""" 
    # Make sure the model is in evaluation mode.
    model.eval()
    # We do not need to maintain intermediate activations while testing.
    accs = []
    with torch.no_grad():
        
        # Loop over test data.
        for features, target in data_loader:
          
            # Forward pass.
            output = model(features.to(device))
            
            # Get the label corresponding to the highest predicted probability.
            pred = output.argmax(dim=1, keepdim=True)
            
            # Count number of correct predictions.
            correct = pred.cpu().eq(target.view_as(pred)).sum().item()
            total = pred.shape[0]
            accs.append(correct/total)

    # Print test accuracy.
    percent = 100. * np.mean(accs)
    st = np.std(accs)
    return percent, st

In [ ]:
def train(model, criterion, data_loader, test_data_loader, optimizer, num_epochs, filename):
    """Simple training loop for a PyTorch model.""" 
    
    # Make sure model is in training mode.
    model.train()
    
    # Move model to the device (CPU or GPU).
    model.to(device)
    
    # Exponential moving average of the loss.
    ema_loss = None
    
    best_acc = 0

    print('----- Training Loop -----')
    # Loop over epochs.
    for epoch in range(num_epochs):
        
        # Loop over data.
        for batch_idx, (features, target) in enumerate(data_loader):

            # Forward pass.
            output = model(features.to(device))
            loss = criterion(output.to(device), target.to(device))

            # Backward pass.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # NOTE: It is important to call .item() on the loss before summing.
            if ema_loss is None:
                ema_loss = loss.item()
            else:
                ema_loss += (loss.item() - ema_loss) * 0.01 

        # Print out progress the end of epoch.
        print('----- Model Evaluation -----')
        print('Epoch: {}/{} \tTrain Loss: {:.6f}'.format(epoch+1,num_epochs, ema_loss))
        train_a, train_st = test(model,data_loader)
        test_a, test_st = test(model,test_data_loader)
        print(f'Train accuracy: ({train_a:.2f}%) with std:({train_st:.2f})')
        print(f'Test accuracy: ({test_a:.2f}%) with std:({test_st:.2f})')
        if test_a > best_acc:
            best_acc = test_a
            torch.save(model.state_dict(), filename+".pth")
            
    checkpoint = torch.load(filename+".pth")
    model.load_state_dict(checkpoint)
    print("------")
    test_a, test_st = test(model,test_data_loader)
    print(f'Final test accuracy: ({test_a:.2f}%) with std:({test_st:.2f})')
    
    return model

In [ ]:
def generate_predictions(model,data_loader):
    model.eval()
    preds=[]
    idx=[]

    print('----- MAKING PREDICTIONS -----')
    # We do not need to maintain intermediate activations while testing.
    with torch.no_grad():
        
        # Loop over test data.
        for features, indices in data_loader:
            
            # Forward pass.
            output = model(features.to(device))
            
            # Get the label corresponding to the highest predicted probability.
            pred = output.argmax(dim=1, keepdim=True)
            for p,ind in zip(pred,indices):
                idx.append(ind)
                preds.append(p.item())

    return preds,idx

def map_to_classes(n):
    return train_data.classes[n]

In [ ]:
# def test(model, data_loader) :
#   ####Measures the accuracy of a model on a data set."''''
#   # Make sure the model is in evaluation mode.
#   model.eval()
#   correct=0
#   total =0
#   print('-----Model Evaluation-----')
#   # We do not need to maintain intermediate activations while testing.
#   with torch.no_grad():
#   # Loop over test data.
#     for features, target in data_loader:
#       # Forward pass.
#       output = model (features.to(device))
#       # Get the label corresponding to the highest predicted probability.
#       pred = output.argmax(dim=1, keepdim=True)
#       # Count number of correct predictions.
#       total +=len(features)
#       correct += pred.cpu().eq(target.view_as (pred)) .sum().item()
#   # Print test accuracy.
#   percent = 100. * correct / total
#   print(f'Test accuracy: {correct} / {total} ({percent:.0f}%) ')
#   torch.save (model.state_dict(),'model.ckpt')
#   return percent

In [ ]:
# from torchvision.models.efficientnet import efficientnet_b0
# efficientnet = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=True)
# efficientnet.get_parameter

In [ ]:
# resnet152 = models.vgg19(pretrained=True)
# resnet152.get_parameter

In [ ]:
def get_resnext(model_name):
    model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
    model.avg_pool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
    model.last_linear = nn.Linear(in_features=2048, out_features=5, bias=True)

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = AdamHD(model.parameters(), lr=1e-4, hypergrad_lr=1e-9)

    return model, criterion, optimizer

In [ ]:
preds = []

In [ ]:
model1, criterion, optimizer = get_resnext(MODEL_NAME1)

model1 = train(model1, criterion, train_loader, valid_loader, optimizer, num_epochs=NUM_EPOCHS1, filename="resnext")

tta_model = tta.ClassificationTTAWrapper(model1, tta.aliases.five_crop_transform(DIM_TEST_2,DIM_TEST_2))
predictions, _ = generate_predictions(tta_model,test_loader)
preds.append(predictions)

----- Training Loop -----


/usr/local/lib/python3.7/dist-packages/hypergrad/adam_hd.py:82: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1050.)
  exp_avg.mul_(beta1).add_(1 - beta1, grad)


----- Model Evaluation -----
Epoch: 1/5 	Train Loss: 0.830860
Train accuracy: (75.65%) with std:(0.15)
Test accuracy: (74.94%) with std:(0.15)
----- Model Evaluation -----
Epoch: 2/5 	Train Loss: 0.753259
Train accuracy: (79.97%) with std:(0.14)
Test accuracy: (78.11%) with std:(0.15)
----- Model Evaluation -----
Epoch: 3/5 	Train Loss: 0.556900
Train accuracy: (81.03%) with std:(0.14)
Test accuracy: (79.05%) with std:(0.13)
----- Model Evaluation -----
Epoch: 4/5 	Train Loss: 0.602112
Train accuracy: (81.44%) with std:(0.14)
Test accuracy: (81.60%) with std:(0.15)
----- Model Evaluation -----
Epoch: 5/5 	Train Loss: 0.520158
Train accuracy: (83.42%) with std:(0.14)
Test accuracy: (81.69%) with std:(0.14)
------
Final test accuracy: (83.27%) with std:(0.11)
----- MAKING PREDICTIONS -----


In [ ]:
final_predictions = np.mean(preds,axis=0)
final_predictions.shape

(3774,)

In [ ]:
ss = pd.DataFrame({
    "Category": final_predictions,
    "Id": test_data.indices
})
ss.head()

,Category,Id
0,2.0,test-img-3031.jpg
1,2.0,test-img-3288.jpg
2,2.0,test-img-2448.jpg
3,2.0,test-img-3751.jpg
4,2.0,test-img-2902.jpg


In [ ]:


ss["Category"] = predictions
ss["Category"] = ss["Category"].apply(map_to_classes)
ss.head()



,Category,Id
0,cmd,test-img-3031.jpg
1,cmd,test-img-3288.jpg
2,cmd,test-img-2448.jpg
3,cmd,test-img-3751.jpg
4,cmd,test-img-2902.jpg


In [ ]:


ss.to_csv("submit15.csv",index=False)

